In [ ]:
import numpy as np
import pandas as pd
from math import pi
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.models import Model
from keras.layers import GRU, Dense, Dropout, concatenate, Input
from keras.optimizers import RMSprop
np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [3]:
data = pd.read_csv('./historic_data.csv')
data['actual'] = data['rides']
train_cut = int(len(data) * 0.8)
validate_cut = int(len(data) * 0.9)

In [4]:
X, y = data.iloc[:, data.columns != 'rides'], data.iloc[:, data.columns == 'rides'].iloc[:, 0]
X.head()

,date,hour,month,day_of_week,sunrise,icon,precip_prob,temperature,humidity,wind_speed,actual
0,2013-06-01,0,6,7,0,clear,0.01,77.65,0.61,2.06,152
1,2013-06-01,1,6,7,0,clear,0.01,75.62,0.67,1.93,102
2,2013-06-01,2,6,7,0,clear,0.01,74.72,0.70,2.31,67
3,2013-06-01,3,6,7,0,clear,0.01,73.32,0.76,2.16,41
4,2013-06-01,4,6,7,0,clear,0.01,72.42,0.79,1.93,16


In [5]:
class YearExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.applymap(lambda x: float(pd.to_datetime(x).year))

class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.loc[:, self.columns]

class CustomTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, func):
        self.func = func
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return self.func(X)

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion([
        ('cat', Pipeline([
            ('cat_selector', ColumnSelector(['hour', 'day_of_week', 'month'])),
            ('cat_encoder', OneHotEncoder(sparse=False))
        ])),
        ('year', Pipeline([
            ('year_selector', ColumnSelector(['date'])),
            ('year_extractor', YearExtractor()),
        ])),
        ('int', Pipeline([
            ('int_selector', ColumnSelector(['sunrise', 'precip_prob', 'temperature', 'humidity', 'wind_speed', 'actual'])),
        ])),
    ])),
    ('scaler', MinMaxScaler())
])

In [7]:
pipeline.fit(X.iloc[0:train_cut, :])
ft_X = pipeline.transform(X)

C:\Users\Brian\Anaconda3\envs\citibike-dashboard\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
scaler = MinMaxScaler()
scaler.fit(np.array(y[0:train_cut]).reshape(-1, 1))

min_max_y = scaler.transform(np.array(y).reshape(-1, 1))

C:\Users\Brian\Anaconda3\envs\citibike-dashboard\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [171]:
def generator(predictors, response, start, stop, lookback, lag, batch_size=2, shuf=True):
    index = start
    order = [i for i in range(start, stop+1)]
    order_index = 0
    if shuf:
        shuffle(order)
    while True:
        if index + batch_size > stop:
            samples = np.zeros((stop-index, 1, lookback))
            meta = np.zeros((stop-index, len(predictors[0])-1))
            targets = np.zeros(stop-index)
        else:
            samples = np.zeros((batch_size, 1, lookback))
            meta = np.zeros((batch_size, len(predictors[0])-1))
            targets = np.zeros(batch_size)
        for i in range(samples.shape[0]):
            samples[i] = predictors[order[order_index]-lookback-lag:order[order_index]-lag, -1].reshape(1, -1)
            meta[i] = predictors[order[order_index], :-1]
            targets[i] = response[order[order_index]]
            order_index += 1

        index += batch_size
        if index >= stop:
            index = start
            order = [i for i in range(start, stop+1)]
            order_index = 0
            if shuf:
                shuffle(order)
        
        yield [samples,meta], targets

In [172]:
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=20
    ),
    ModelCheckpoint(
        filepath='my_model9.h5',
        monitor='val_loss',
        save_best_only=True
    )
]
        
lookback = 14*24 # seven days
lag = 2*24 # one day
batch_size = 64
train_steps = (train_cut - lookback - lag) // batch_size + 1
val_steps = (validate_cut - train_cut) // batch_size + 1


train_gen = generator(ft_X, min_max_y, lookback+lag, train_cut, lookback, lag, batch_size)
validate_gen = generator(ft_X, min_max_y, train_cut, validate_cut, lookback, lag, batch_size)

In [158]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D

historical = Input(shape=(1, lookback), name='historical')
historical1 = GRU(64, dropout=0.5, recurrent_dropout=0.5, input_shape=(None, ))(historical)

weather = Input(shape=(49, ), name='weather')
weather1 = Dense(64, activation='relu')(weather)
weather1 = Dropout(0.5)(weather1)

concat = concatenate([historical1, weather1])
concat1 = Dense(128, activation='relu')(concat)
concat1 = Dropout(0.5)(concat1)
concat1 = Dense(64, activation='relu')(concat1)
concat1 = Dropout(0.5)(concat1)
output = Dense(1)(concat1)

model = Model([historical, weather], output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
weather (InputLayer)            (None, 49)           0                                            
__________________________________________________________________________________________________
historical (InputLayer)         (None, 1, 336)       0                                            
__________________________________________________________________________________________________
dense_57 (Dense)                (None, 64)           3200        weather[0][0]                    
__________________________________________________________________________________________________
gru_13 (GRU)                    (None, 64)           76992       historical[0][0]                 
__________________________________________________________________________________________________
dropout_28

In [159]:
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=75, 
                    validation_data=validate_gen, validation_steps=val_steps,
                    callbacks=callbacks_list)

Epoch 1/75
492/492 [==============================] - 5s 11ms/step - loss: 0.0148 - mean_absolute_error: 0.0838 - val_loss: 0.0159 - val_mean_absolute_error: 0.0901
Epoch 2/75
492/492 [==============================] - 3s 7ms/step - loss: 0.0069 - mean_absolute_error: 0.0585 - val_loss: 0.0109 - val_mean_absolute_error: 0.0734
Epoch 3/75
492/492 [==============================] - 3s 7ms/step - loss: 0.0056 - mean_absolute_error: 0.0525 - val_loss: 0.0106 - val_mean_absolute_error: 0.0730
Epoch 4/75
492/492 [==============================] - 3s 6ms/step - loss: 0.0050 - mean_absolute_error: 0.0495 - val_loss: 0.0079 - val_mean_absolute_error: 0.0605
Epoch 5/75
492/492 [==============================] - 3s 6ms/step - loss: 0.0044 - mean_absolute_error: 0.0470 - val_loss: 0.0088 - val_mean_absolute_error: 0.0632
Epoch 6/75
492/492 [==============================] - 3s 6ms/step - loss: 0.0042 - mean_absolute_error: 0.0455 - val_loss: 0.0074 - val_mean_absolute_error: 0.0587
Epoch 7/75
492/

In [188]:
from sklearn.metrics import mean_squared_error
from keras.models import load_model

mmodel = load_model('my_model9.h5')

new_gen = generator(ft_X, min_max_y, train_cut, validate_cut, lookback, lag, batch_size, shuf=False)
predictions = model.predict_generator(new_gen, steps=val_steps)
mean_squared_error(scaler.inverse_transform(predictions), y[train_cut:validate_cut])

348428635526023.4

In [ ]:
new_gen = generator(ft_X, y, validate_cut, len(data), lookback, lag, batch_size, shuf=False)
predictions = model.predict_generator(new_gen, steps=val_steps)
mean_squared_error(predictions, y[validate_cut:])

In [180]:
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=20
    ),
    ModelCheckpoint(
        filepath='my_model11.h5',
        monitor='val_loss',
        save_best_only=True
    )
]
        
lookback = 21*24 # seven days
lag = 2*24 # one day
batch_size = 64
train_steps = (train_cut - lookback - lag) // batch_size + 1
val_steps = (validate_cut - train_cut) // batch_size + 1

train_gen = generator(ft_X, y, lookback+lag, train_cut, lookback, lag, batch_size)
validate_gen = generator(ft_X, y, train_cut, validate_cut, lookback, lag, batch_size)

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D

historical = Input(shape=(1, lookback), name='historical')
historical1 = GRU(128, dropout=0.5, recurrent_dropout=0.5, input_shape=(None, ))(historical)
historical1 = Dense(256, activation='relu')(historical1)
historical1 = Dropout(0.5)(historical1)

weather = Input(shape=(49, ), name='weather')
weather1 = Dense(64, activation='relu')(weather)
weather1 = Dropout(0.5)(weather1)

concat = concatenate([historical1, weather1])
concat1 = Dense(128, activation='relu')(concat)
concat1 = Dropout(0.5)(concat1)
concat1 = Dense(64, activation='relu')(concat1)
concat1 = Dropout(0.5)(concat1)
output = Dense(1)(concat1)

model = Model([historical, weather], output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=100, 
                    validation_data=validate_gen, validation_steps=val_steps,
                    callbacks=callbacks_list)

Epoch 1/100
490/490 [==============================] - 8s 15ms/step - loss: 682392.7240 - mean_absolute_error: 514.1339 - val_loss: 587883.3019 - val_mean_absolute_error: 503.0111
Epoch 2/100
490/490 [==============================] - 4s 8ms/step - loss: 331320.1240 - mean_absolute_error: 367.2021 - val_loss: 476153.6919 - val_mean_absolute_error: 454.7355
Epoch 3/100
490/490 [==============================] - 4s 8ms/step - loss: 296640.1428 - mean_absolute_error: 342.9276 - val_loss: 442192.5753 - val_mean_absolute_error: 437.0235
Epoch 4/100
490/490 [==============================] - 4s 8ms/step - loss: 278983.6530 - mean_absolute_error: 333.3304 - val_loss: 455082.0350 - val_mean_absolute_error: 450.8656
Epoch 5/100
490/490 [==============================] - 4s 8ms/step - loss: 263464.7562 - mean_absolute_error: 323.4791 - val_loss: 491753.3197 - val_mean_absolute_error: 461.3687
Epoch 6/100
490/490 [==============================] - 4s 8ms/step - loss: 259085.8584 - mean_absolute_e

490/490 [==============================] - 4s 8ms/step - loss: 196435.5267 - mean_absolute_error: 274.2242 - val_loss: 200480.7185 - val_mean_absolute_error: 278.8434
Epoch 47/100
490/490 [==============================] - 4s 8ms/step - loss: 197109.7753 - mean_absolute_error: 272.6228 - val_loss: 204682.2665 - val_mean_absolute_error: 274.7608
Epoch 48/100
490/490 [==============================] - 4s 8ms/step - loss: 197978.6647 - mean_absolute_error: 273.7065 - val_loss: 214842.1845 - val_mean_absolute_error: 286.3170
Epoch 49/100
490/490 [==============================] - 4s 8ms/step - loss: 194697.7019 - mean_absolute_error: 271.0265 - val_loss: 193714.5939 - val_mean_absolute_error: 274.8369
Epoch 50/100
490/490 [==============================] - 4s 8ms/step - loss: 194138.8524 - mean_absolute_error: 269.6350 - val_loss: 202435.9020 - val_mean_absolute_error: 276.9298
Epoch 51/100
490/490 [==============================] - 4s 8ms/step - loss: 192966.9346 - mean_absolute_error: 26

490/490 [==============================] - 4s 8ms/step - loss: 186942.6469 - mean_absolute_error: 266.7967 - val_loss: 191787.9625 - val_mean_absolute_error: 265.7032
Epoch 92/100
490/490 [==============================] - 4s 8ms/step - loss: 182464.8916 - mean_absolute_error: 261.7419 - val_loss: 198536.0418 - val_mean_absolute_error: 275.5772
Epoch 93/100
490/490 [==============================] - 4s 8ms/step - loss: 184372.5181 - mean_absolute_error: 262.8973 - val_loss: 195068.3933 - val_mean_absolute_error: 272.4435
Epoch 94/100
490/490 [==============================] - 4s 8ms/step - loss: 183119.4307 - mean_absolute_error: 262.9034 - val_loss: 186720.7375 - val_mean_absolute_error: 265.4643
Epoch 95/100
490/490 [==============================] - 4s 8ms/step - loss: 182852.9956 - mean_absolute_error: 262.9579 - val_loss: 190567.2795 - val_mean_absolute_error: 266.8952
Epoch 96/100
490/490 [==============================] - 4s 8ms/step - loss: 180043.1581 - mean_absolute_error: 26

In [193]:
mmodel = load_model('my_model11.h5')

new_gen = generator(ft_X, y, train_cut, validate_cut, lookback, lag, batch_size, shuf=False)
predictions = mmodel.predict_generator(new_gen, steps=val_steps)
mean_squared_error(predictions, y[train_cut:validate_cut])

179568.40411785862

In [194]:
new_gen = generator(ft_X, y, validate_cut, len(data), lookback, lag, batch_size, shuf=False)
predictions = mmodel.predict_generator(new_gen, steps=val_steps)
mean_squared_error(predictions, y[validate_cut:])

254101.35173929893

In [236]:
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=10
    ),
    ModelCheckpoint(
        filepath='my_model14all.h5',
        monitor='val_loss',
        save_best_only=True
    )
]
        
lookback = 90*24 # seven days
lag = 2*24 # one day
batch_size = 128
train_steps = (train_cut - lookback - lag) // batch_size + 1
val_steps = (validate_cut - train_cut) // batch_size + 1

train_gen = generator(ft_X, y, lookback+lag, validate_cut, lookback, lag, batch_size)
validate_gen = generator(ft_X, y, train_cut, validate_cut, lookback, lag, batch_size)

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D

historical = Input(shape=(1, lookback), name='historical')
historical1 = GRU(16, dropout=0.5, recurrent_dropout=0.5, input_shape=(None, ))(historical)

weather = Input(shape=(49, ), name='weather')
weather1 = Dense(16, activation='relu')(weather)
weather1 = Dropout(0.5)(weather1)

concat = concatenate([historical1, weather1])
concat1 = Dense(32, activation='relu')(concat)
concat1 = Dropout(0.5)(concat1)
concat1 = Dense(32, activation='relu')(concat1)
concat1 = Dropout(0.5)(concat1)
output = Dense(1)(concat1)

model = Model([historical, weather], output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=75,
                     validation_data=validate_gen, validation_steps=val_steps,
                     callbacks=callbacks_list)

Epoch 1/75
232/232 [==============================] - 12s 51ms/step - loss: 2647560.4725 - mean_absolute_error: 1119.0565 - val_loss: 2271739.0182 - val_mean_absolute_error: 999.8818
Epoch 2/75
232/232 [==============================] - 3s 12ms/step - loss: 916912.8461 - mean_absolute_error: 626.6592 - val_loss: 1096489.8425 - val_mean_absolute_error: 687.2201
Epoch 3/75
232/232 [==============================] - 3s 12ms/step - loss: 699748.0468 - mean_absolute_error: 531.8061 - val_loss: 803007.7762 - val_mean_absolute_error: 580.5482
Epoch 4/75
232/232 [==============================] - 3s 12ms/step - loss: 585353.7774 - mean_absolute_error: 491.0125 - val_loss: 761951.9875 - val_mean_absolute_error: 584.3167
Epoch 5/75
232/232 [==============================] - 3s 12ms/step - loss: 542931.7429 - mean_absolute_error: 464.3577 - val_loss: 617971.7784 - val_mean_absolute_error: 520.0919
Epoch 6/75
232/232 [==============================] - 3s 13ms/step - loss: 522199.4379 - mean_absolu

232/232 [==============================] - 3s 11ms/step - loss: 393467.4135 - mean_absolute_error: 389.4413 - val_loss: 265451.7428 - val_mean_absolute_error: 342.2583
Epoch 47/75
232/232 [==============================] - 3s 13ms/step - loss: 395960.4228 - mean_absolute_error: 390.9776 - val_loss: 267444.1800 - val_mean_absolute_error: 345.1954
Epoch 48/75
232/232 [==============================] - 3s 12ms/step - loss: 394822.9463 - mean_absolute_error: 390.7807 - val_loss: 277899.9963 - val_mean_absolute_error: 343.8997
Epoch 49/75
232/232 [==============================] - 3s 12ms/step - loss: 402372.2999 - mean_absolute_error: 394.9073 - val_loss: 322462.1929 - val_mean_absolute_error: 380.7220
Epoch 50/75
232/232 [==============================] - 3s 12ms/step - loss: 409008.5087 - mean_absolute_error: 397.2880 - val_loss: 278677.8664 - val_mean_absolute_error: 349.3607
Epoch 51/75
232/232 [==============================] - 3s 12ms/step - loss: 391394.7297 - mean_absolute_error: 3

In [237]:
# mmodel = load_model('my_model14all.h5')

new_gen = generator(ft_X, y, train_cut, validate_cut, lookback, lag, batch_size, shuf=False)
predictions = model.predict_generator(new_gen, steps=val_steps)
mean_squared_error(predictions, y[train_cut:validate_cut])

216115.5839459362

In [239]:
new_gen = generator(ft_X, y, validate_cut, len(data), lookback, lag, batch_size, shuf=False)
predictions = model.predict_generator(new_gen, steps=val_steps)
mean_squared_error(predictions, y[validate_cut:])

362357.8857199827

In [201]:
mmodel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
historical (InputLayer)         (None, 1, 504)       0                                            
__________________________________________________________________________________________________
gru_20 (GRU)                    (None, 128)          243072      historical[0][0]                 
__________________________________________________________________________________________________
weather (InputLayer)            (None, 49)           0                                            
__________________________________________________________________________________________________
dense_87 (Dense)                (None, 256)          33024       gru_20[0][0]                     
__________________________________________________________________________________________________
dense_88 (